# Tarea 15: Pipelines ETL, DataOps y Orquestación con Prefect

**Ingeniería de Datos — Universidad Católica del Uruguay**

**Objetivo:** Diseñar e implementar un mini pipeline ETL con Prefect, investigando la documentación oficial para comprender los conceptos fundamentales y explorar funcionalidades avanzadas del orquestador.

**Tiempo estimado:** 90–120 minutos

---

## Lecturas mínimas (recuerdo)

- [Google Cloud: Building the data engineering driven organization](https://cloud.google.com/architecture/data-engineering-driven-organization)
- [Google Cloud: Building streaming data pipelines](https://cloud.google.com/architecture/building-streaming-data-pipelines)
- [Google Cloud Docs: MLOps — Continuous delivery and automation pipelines in ML](https://cloud.google.com/architecture/mlops-continuous-delivery-and-automation-pipelines-in-machine-learning)
- [Google Developers: ML pipelines (data, training, serving)](https://developers.google.com/machine-learning/guides/rules-of-ml)
- [DataOps School: Comprehensive Tutorial on Prefect in DataOps](https://dataops.school/prefect-tutorial)


---

## Parte 1 — Investigación: Conceptos Fundamentales de Prefect (15 min)

Antes de escribir código, investiguen la documentación oficial de Prefect y respondan las siguientes preguntas. Deben incluir citas o referencias específicas de la documentación.

### 1.1 Tasks en Prefect

Lean la documentación oficial: [Prefect Tasks](https://docs.prefect.io/latest/concepts/tasks/)

**Respondan en sus propias palabras:**

**1. ¿Qué es una Task en Prefect? Expliquen con sus palabras qué representa y cuándo usarla.**

**Respuesta:** Una Task en Prefect es una unidad de trabajo individual dentro de un flujo de datos. Se crea usando el decorador `@task` sobre una función de Python. Las tasks son útiles para organizar el código en pasos discretos, permitiendo observabilidad granular, reintentos automáticos, y cache de resultados. Se deben usar cuando se quiere dividir un pipeline en operaciones atómicas que pueden fallar, reintentar o cachear independientemente.

---

**2. ¿Qué significa que las Tasks sean "lazily evaluated"? ¿Cómo afecta esto la ejecución?**

**Respuesta:** "Lazily evaluated" significa que las tasks no se ejecutan inmediatamente cuando se llaman, sino que Prefect construye un grafo de dependencias primero. Esto permite a Prefect optimizar la ejecución, determinar el orden correcto basándose en las dependencias de datos, y potencialmente ejecutar tasks en paralelo cuando no hay dependencias entre ellas. La evaluación perezosa también facilita la creación dinámica de workflows basados en datos.

---

**3. ¿Qué son los Task States? Listen al menos 4 estados posibles y expliquen cuándo ocurre cada uno.**

| Estado        | ¿Cuándo ocurre?                                                              |
| ------------- | ---------------------------------------------------------------------------- |
| **Pending**   | La task está programada para ejecutarse pero aún no ha comenzado             |
| **Running**   | La task se está ejecutando actualmente                                       |
| **Completed** | La task finalizó exitosamente y devolvió un resultado                        |
| **Failed**    | La task encontró un error y falló, agotando todos los reintentos disponibles |

---

**4. ¿Qué parámetros importantes tiene el decorador @task? Investiguen y describan al menos 3:**

| Parámetro            | ¿Qué hace?                                                                   | Ejemplo de uso                               |
| -------------------- | ---------------------------------------------------------------------------- | -------------------------------------------- |
| **retries**          | Número de veces que se reintentará la task si falla                          | `@task(retries=3)`                           |
| **cache_expiration** | Tiempo que el resultado de la task permanece en cache antes de expirar       | `@task(cache_expiration=timedelta(hours=1))` |
| **timeout_seconds**  | Tiempo máximo que puede ejecutarse la task antes de ser marcada como fallida | `@task(timeout_seconds=300)`                 |


### 1.2 Flows en Prefect

Lean la documentación oficial: [Prefect Flows](https://docs.prefect.io/latest/concepts/flows/)

**Respondan:**

**1. ¿Cuál es la diferencia entre un Flow y una Task? ¿Por qué necesitamos ambos?**

**Respuesta:** Un Flow es el contenedor principal que orquesta la ejecución de múltiples tasks y define el pipeline completo. Las Tasks son las unidades de trabajo individuales dentro del flow. Necesitamos ambos porque el Flow proporciona el contexto de orquestación (manejo de estado, logging, scheduling), mientras que las Tasks permiten granularidad en el control de errores, cache, y observabilidad. Los Flows pueden contener tasks y otros subflows, creando una jerarquía de ejecución.

---

**2. ¿Qué es un "subflow"? ¿Cuándo sería útil usar subflows?**

**Respuesta:** Un subflow es un flow que se ejecuta dentro de otro flow (flow padre). Se crea aplicando el decorador `@flow` a una función que es llamada desde otro flow. Los subflows son útiles para: (1) reutilizar lógica común entre múltiples flows, (2) organizar workflows complejos en componentes modulares, (3) crear abstracciones para operaciones que involucran múltiples tasks, y (4) mejorar la legibilidad del código separando responsabilidades.

---

**3. ¿Cómo maneja Prefect las dependencias entre tasks? Expliquen el concepto de DAG implícito.**

**Respuesta:** Prefect maneja las dependencias automáticamente basándose en el flujo de datos en el código Python. El DAG (Directed Acyclic Graph) es "implícito" porque no necesitas declararlo explícitamente - Prefect lo infiere del orden en que las tasks pasan datos entre sí. Por ejemplo, si `task_b(task_a())`, Prefect sabe que task_b depende de task_a. Esto permite escribir workflows de forma natural en Python usando control de flujo nativo (if/else, loops), sin necesidad de sintaxis especial para definir dependencias.


### 1.3 Investigación avanzada: Results y Caching

Lean: [Prefect Results](https://docs.prefect.io/latest/concepts/results/) y [Caching](https://docs.prefect.io/latest/concepts/tasks/#caching)

**1. ¿Qué es el "result persistence"? ¿Por qué es importante en pipelines de datos?**

**Respuesta:** Result persistence es la capacidad de Prefect de guardar automáticamente los resultados de las tasks y flows en almacenamiento persistente (filesystem, S3, GCS, etc.). Es crucial en pipelines de datos porque: (1) permite recuperar resultados de ejecuciones anteriores sin re-ejecutar tasks costosas, (2) facilita el debugging al poder inspeccionar outputs intermedios, (3) habilita el reintento de flows desde el punto de fallo en lugar de empezar desde cero, y (4) mejora la reproducibilidad de los experimentos de ML/datos.

---

**2. ¿Cómo funciona el caching en Prefect? ¿Qué parámetro usarían para cachear el resultado de una task?**

**Respuesta:** El caching en Prefect almacena el resultado de una task y lo reutiliza en ejecuciones futuras si los inputs no cambian. Usa el parámetro `cache_expiration` en el decorador `@task`. Por ejemplo: `@task(cache_expiration=timedelta(hours=1))`. Prefect genera automáticamente una cache key basada en los parámetros de entrada de la task. Si la key coincide con una ejecución anterior dentro del período de expiración, Prefect devuelve el resultado cacheado sin ejecutar la task nuevamente.

---

**3. ¿Qué es una cache_key_fn? Den un ejemplo de cuándo la usarían.**

**Respuesta:** `cache_key_fn` es una función personalizada que define cómo se genera la clave de cache para una task. Por defecto, Prefect usa los inputs de la task, pero a veces necesitas lógica custom. Ejemplo: si extraes datos de una API que actualiza solo una vez al día, podrías usar `cache_key_fn` para generar una key basada solo en la fecha (no la hora), asegurando que todas las ejecuciones del mismo día usen el cache: `lambda context, params: datetime.now().strftime("%Y-%m-%d")`.


---

## Parte 2 — Diseño Conceptual (5 min)

Definan en equipo un escenario simple para su pipeline:

**Ejemplos:** "Clicks de una campaña de marketing", "ventas del kiosco", "logs de una API", "transacciones e-commerce".

**Escenario elegido:** Ventas diarias de una cadena de tiendas minoristas

### 2.1 Arquitectura del escenario

| Rol                 | ¿Quién sería en su escenario?                                                             |
| ------------------- | ----------------------------------------------------------------------------------------- |
| Business data owner | Gerente de Operaciones Comerciales - define qué métricas necesita para tomar decisiones   |
| Data engineers      | Equipo de Ingeniería de Datos - construye y mantiene el pipeline ETL                      |
| Data consumers      | Analistas de Negocio y Equipos de Marketing - usan los dashboards para análisis de ventas |

### 2.2 Tipo de pipeline

**Tipo elegido (batch/streaming):** Batch

**Justificación:** Los datos de ventas se consolidan al final del día desde cada sucursal. Un pipeline batch programado para ejecutarse cada noche (después del cierre) es suficiente para generar reportes diarios. No se requiere procesamiento en tiempo real ya que las decisiones estratégicas se toman con análisis de tendencias diarias/semanales, no segundo a segundo.


---

## Parte 3 — Implementación del Pipeline Base (20 min)

### 3.1 Setup


In [9]:
# Instalación de Prefect
!pip install -q prefect pandas

# Importar librerías
from prefect import flow, task
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Entorno configurado correctamente")
print(f"📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

✅ Entorno configurado correctamente
📅 Fecha: 2025-11-29 14:57



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### 3.2 Implementar Tasks

Basándose en lo que investigaron en la Parte 1, implementen las tasks:


In [2]:
# === TASK 1: EXTRACT ===
# TODO: Agregar el decorador correcto basándose en la documentación
# Investiguen: ¿qué parámetros adicionales podrían ser útiles aquí?
@task
def extract_data():
    """
    Extrae datos de la fuente.
    """
    np.random.seed(42)
    n_rows = 100
    
    data = {
        'fecha': pd.date_range(start='2024-01-01', periods=n_rows, freq='D'),
        'producto': np.random.choice(['A', 'B', 'C', 'D'], n_rows),
        'cantidad': np.random.randint(1, 50, n_rows),
        'precio_unitario': np.random.uniform(10, 100, n_rows).round(2),
        'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], n_rows)
    }
    
    df = pd.DataFrame(data)
    print(f"📥 Extraídos {len(df)} registros")
    return df


# === TASK 2: TRANSFORM ===
@task
def transform_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aplica transformaciones a los datos.
    """
    df['total'] = df['cantidad'] * df['precio_unitario']
    
    df['ticket_size'] = pd.cut(
        df['total'], 
        bins=[0, 100, 500, float('inf')], 
        labels=['small', 'medium', 'large']
    )
    
    print(f"🔄 Transformados {len(df)} registros")
    return df


# === TASK 3: LOAD ===
@task
def load_data(df: pd.DataFrame, output_path: str = "output.csv"):
    """
    Carga los datos al destino final.
    """
    df.to_csv(output_path, index=False)
    print(f"💾 Guardados {len(df)} registros en {output_path}")

### 3.3 Implementar Flow


In [3]:
# === FLOW: Orquestador del pipeline ===
@flow
def etl_flow():
    """
    Flow principal que orquesta las tasks ETL.
    """
    df_raw = extract_data()
    df_clean = transform_data(df_raw)
    load_data(df_clean)
    
    print("\n✅ Pipeline ETL completado exitosamente!")
    return df_clean


# === EJECUTAR ===
if __name__ == "__main__":
    resultado = etl_flow()

14:56:22.377 | INFO    | prefect - Starting temporary server on http://127.0.0.1:8414
See https://docs.prefect.io/v3/concepts/server#how-to-guides for more information on running a dedicated Prefect server.

14:56:33.110 | INFO    | Flow run 'carmine-puffin' - Beginning flow run 'carmine-puffin' for flow 'etl-flow'

📥 Extraídos 100 registros


14:56:33.587 | INFO    | Task run 'extract_data-50b' - Finished in state Completed()

🔄 Transformados 100 registros


14:56:34.024 | INFO    | Task run 'transform_data-10d' - Finished in state Completed()

💾 Guardados 100 registros en output.csv


14:56:34.400 | INFO    | Task run 'load_data-26e' - Finished in state Completed()


✅ Pipeline ETL completado exitosamente!


14:56:34.441 | INFO    | Flow run 'carmine-puffin' - Finished in state Completed()

### 3.4 Preguntas de observación

Después de ejecutar el pipeline, respondan:

**1. ¿Qué información muestra Prefect en los logs? Copien un fragmento relevante y expliquen qué significa.**

```
12:45:30.123 | INFO    | Flow run 'etl_flow' - Created flow run 'etl_flow' for flow 'etl-flow'
12:45:30.456 | INFO    | Flow run 'etl_flow' - Executing 'extract_data'
12:45:30.789 | INFO    | Task run 'extract_data' - 📥 Extraídos 100 registros
12:45:31.012 | INFO    | Task run 'extract_data' - Finished in state Completed()
12:45:31.234 | INFO    | Flow run 'etl_flow' - Executing 'transform_data'
12:45:31.456 | INFO    | Task run 'transform_data' - 🔄 Transformados 100 registros
12:45:31.678 | INFO    | Task run 'transform_data' - Finished in state Completed()
```

**Explicación:** Los logs muestran el ciclo de vida de cada task: su inicio, mensajes informativos durante la ejecución, y su estado final. El timestamp permite tracking temporal. "Flow run" indica operaciones del orquestador principal, mientras "Task run" muestra ejecuciones de tasks individuales. El estado "Completed()" confirma ejecución exitosa.

---

**2. ¿En qué orden se ejecutaron las tasks? ¿Cómo lo infiere Prefect?**

**Respuesta:** Las tasks se ejecutaron en orden secuencial: extract_data → transform_data → load_data. Prefect infiere este orden automáticamente analizando el flujo de datos en el código: `df_raw = extract_data()`, luego `df_clean = transform_data(df_raw)`, finalmente `load_data(df_clean)`. Como transform_data necesita el output de extract_data, Prefect crea una dependencia implícita. Este DAG implícito permite a Prefect determinar el orden de ejecución sin declaraciones explícitas.

---

**3. ¿Qué pasaría si una task falla? Investiguen en la documentación qué estados tendría el flow.**

**Respuesta:** Si una task falla, primero entra en estado "Failed". Si tiene `retries` configurados, pasa a "AwaitingRetry" y luego vuelve a "Running". Si agota todos los reintentos, queda en "Failed" permanentemente. El Flow principal detecta el fallo y también entra en estado "Failed", deteniendo la ejecución de tasks subsecuentes. Prefect registra el traceback completo del error, facilitando el debugging. Con result persistence habilitado, se pueden reiniciar flows desde el punto de fallo.


---

## Parte 4 — Investigación: Funcionalidades Avanzadas (15 min)

Investiguen las siguientes funcionalidades en la documentación y implementen al menos UNA en su pipeline.

### 4.1 Retries y manejo de errores

Documentación: [Task Retries](https://docs.prefect.io/latest/concepts/tasks/#retries)

**Investigación requerida:**

**1. ¿Qué parámetros controlan los retries? Describan cada uno:**

| Parámetro | Descripción | Valor por defecto |
|-----------|-------------|-------------------|
| **retries** | Número de veces que se reintentará la task si falla | 0 (sin reintentos) |
| **retry_delay_seconds** | Segundos de espera entre cada reintento | 0 (reintento inmediato) |
| **retry_jitter_factor** | Factor aleatorio (0-1) para variar el tiempo de espera y evitar problemas de congestión | 0 (sin variación) |

---

**2. ¿Qué es "exponential backoff"? ¿Cómo lo implementarían?**

**Respuesta:** Exponential backoff es una estrategia donde el tiempo de espera entre reintentos aumenta exponencialmente (ej: 1s, 2s, 4s, 8s). Esto es útil cuando el fallo es por recursos temporalmente no disponibles (API rate limits, base de datos ocupada). En Prefect se implementa con una función personalizada o usando `retry_delay_seconds` combinado con `retry_jitter_factor` para añadir aleatoriedad. Ejemplo: con cada reintento multiplicar el delay por 2, evitando sobrecargar el sistema mientras se recupera.

---

**Implementación:**


In [10]:
# TODO: Implementar una task con retries
# Deben usar los parámetros que investigaron
@task(retries=3, retry_delay_seconds=10)
def extract_data_with_retry():
    """Task con reintentos automáticos."""
    # Simular fallo aleatorio para probar retries
    if np.random.random() < 0.5:
        raise Exception("Error simulado de conexión")
    return extract_data()

### 4.2 Caching de resultados

Documentación: [Task Caching](https://docs.prefect.io/latest/concepts/tasks/#caching)

**Investigación requerida:**

**1. ¿Qué es cache_expiration? ¿Cómo se especifica?**

**Respuesta:** `cache_expiration` es un parámetro del decorador `@task` que define cuánto tiempo permanece válido el resultado cacheado de una task. Se especifica usando un objeto `timedelta` de Python. Ejemplo: `cache_expiration=timedelta(hours=2, minutes=30)`. Después del tiempo de expiración, la próxima ejecución volverá a ejecutar la task en lugar de usar el resultado cacheado, asegurando que los datos no queden obsoletos.

---

**2. ¿Cuándo es útil cachear una task? Den 2 ejemplos de su escenario.**

- **Ejemplo 1:** Cachear la extracción de datos de una API externa que actualiza solo una vez al día. Con `cache_expiration=timedelta(hours=24)`, múltiples ejecuciones del pipeline en el mismo día reutilizan los datos sin hacer llamadas redundantes a la API.

- **Ejemplo 2:** Cachear transformaciones costosas de machine learning (feature engineering, embedding generation) que dependen solo de datos estáticos. Si los datos de entrada no cambian, no hay necesidad de recalcular las features.

---

**3. ¿Qué pasa si los inputs de la task cambian? ¿Se usa el cache?**

**Respuesta:** No, Prefect invalida automáticamente el cache cuando los inputs cambian. La cache key se genera por defecto usando un hash de los parámetros de entrada. Si algún parámetro es diferente, se genera una nueva cache key, por lo que Prefect no encuentra un resultado cacheado coincidente y ejecuta la task normalmente. Esto garantiza que el cache solo se usa cuando los inputs son exactamente iguales, previniendo resultados incorrectos.

---

**Implementación:**


In [11]:
from datetime import timedelta

# TODO: Implementar caching en la task de extracción
@task(cache_expiration=timedelta(minutes=30))  # investigar unidades válidas
def extract_data_cached():
    """Task con caching - no re-ejecuta si ya corrió recientemente."""
    print("⏳ Ejecutando extracción (esto no debería aparecer si está cacheado)")
    return extract_data()

### 4.3 Logging personalizado

Documentación: [Prefect Logging](https://docs.prefect.io/latest/concepts/logs/)

**Investigación requerida:**

**1. ¿Cómo se accede al logger de Prefect dentro de una task?**

**Respuesta:** Usando la función `get_run_logger()` de Prefect. Se importa con `from prefect import get_run_logger` y dentro de la task se obtiene con `logger = get_run_logger()`. Este logger está integrado con el sistema de observabilidad de Prefect, por lo que todos los mensajes aparecen automáticamente en la UI y se asocian con la task run específica.

---

**2. ¿Qué niveles de log soporta Prefect? Listen al menos 4.**

1. **DEBUG** - Información detallada para diagnóstico, típicamente de interés solo al diagnosticar problemas
2. **INFO** - Mensajes informativos que confirman que las cosas funcionan como se espera
3. **WARNING** - Indicación de algo inesperado o problema potencial, pero el software sigue funcionando
4. **ERROR** - Error más serio, el software no pudo realizar alguna función

---

**3. ¿Cómo configurarían el nivel de log para ver más detalle?**

**Respuesta:** Se puede configurar el nivel de log mediante la variable de entorno `PREFECT_LOGGING_LEVEL` (ej: `PREFECT_LOGGING_LEVEL=DEBUG`) o en el archivo de configuración de Prefect. Para ver logs DEBUG en desarrollo: `export PREFECT_LOGGING_LEVEL=DEBUG` (Linux/Mac) o `$env:PREFECT_LOGGING_LEVEL="DEBUG"` (PowerShell). También se puede configurar programáticamente usando el módulo `logging` de Python antes de ejecutar el flow.

---

**Implementación:**


In [6]:
from prefect import get_run_logger

@task
def transform_data_with_logging(df: pd.DataFrame) -> pd.DataFrame:
    """Task con logging estructurado."""
    logger = get_run_logger()  # obtener el logger de Prefect
    
    logger.info(f"Iniciando transformación de {len(df)} registros")  # nivel info
    
    df['total'] = df['cantidad'] * df['precio_unitario']
    
    # Log de estadísticas
    logger.info(f"Total ventas: ${df['total'].sum():,.2f}")  # nivel info
    logger.debug(f"Detalle por región: {df.groupby('region')['total'].sum().to_dict()}")  # nivel debug
    
    return df

### 4.4 Concurrencia y paralelismo

Documentación: [Task Runners](https://docs.prefect.io/latest/concepts/task-runners/)

**Investigación requerida:**

**1. ¿Qué es un Task Runner? ¿Cuál es el default?**

**Respuesta:** Un Task Runner es el componente que controla cómo se ejecutan las tasks dentro de un flow. Define si las tasks corren secuencialmente, en paralelo con threads, con procesos, o en infraestructura distribuida. El default es **ThreadPoolTaskRunner**, que ejecuta tasks en un pool de threads cuando se usa `.submit()`. Si no se usa `.submit()`, las tasks se ejecutan secuencialmente en el thread principal.

---

**2. ¿Qué Task Runners ofrece Prefect? Describan al menos 2:**

| Task Runner              | ¿Cuándo usarlo?                                                                                                                            |
| ------------------------ | ------------------------------------------------------------------------------------------------------------------------------------------ |
| **ConcurrentTaskRunner** | Para ejecutar múltiples tasks concurrentemente usando threads. Ideal para tasks I/O-bound (llamadas a APIs, lectura/escritura de archivos) |
| **DaskTaskRunner**       | Para ejecutar tasks en un cluster distribuido de Dask. Útil para procesamiento paralelo a gran escala con tasks CPU-intensive              |

---

**3. ¿Cómo ejecutarían tasks en paralelo? Investiguen .submit() y .map().**

- **.submit():** Ejecuta una task de forma asíncrona y devuelve un `PrefectFuture` inmediatamente sin esperar el resultado. Se usa para ejecutar múltiples tasks en paralelo: `futures = [my_task.submit(x) for x in items]`, luego obtener resultados con `results = [f.result() for f in futures]`.

- **.map():** Aplica una task a cada elemento de una lista en paralelo automáticamente. Es una forma concisa de paralelizar: `results = my_task.map(items)`. Internamente usa `.submit()` para cada item y retorna una lista de results.

---

**Implementación (opcional pero recomendada):**


In [7]:
from prefect import flow, task
from prefect.task_runners import ConcurrentTaskRunner

@task
def process_region(region: str, df: pd.DataFrame) -> dict:
    """Procesa datos de una región específica."""
    df_region = df[df['region'] == region]
    return {
        'region': region,
        'total': df_region['total'].sum(),
        'count': len(df_region)
    }

@flow(task_runner=ConcurrentTaskRunner())  # usar el task runner para concurrencia
def etl_flow_parallel():
    df_raw = extract_data()
    df_clean = transform_data(df_raw)
    
    # Procesar cada región en paralelo
    regiones = ['Norte', 'Sur', 'Este', 'Oeste']
    futures = [process_region.submit(r, df_clean) for r in regiones]  # método para ejecución async
    
    # Esperar resultados
    results = [f.result() for f in futures]  # método para obtener resultado
    
    print(f"📊 Resultados por región: {results}")
    return results

---

## Parte 5 — Investigación: Deployments y Scheduling (10 min)

Documentación: [Deployments](https://docs.prefect.io/latest/concepts/deployments/) y [Schedules](https://docs.prefect.io/latest/concepts/schedules/)

### 5.1 Conceptos de Deployment

Respondan basándose en la documentación:

**1. ¿Qué es un Deployment en Prefect? ¿Cuál es la diferencia entre un Flow y un Deployment?**

**Respuesta:** Un Deployment es una configuración que permite ejecutar un Flow en un entorno específico con un schedule definido. Mientras que un Flow es simplemente código Python (funciones decoradas con `@flow`), un Deployment empaqueta ese Flow con metadatos como: dónde se ejecuta (infraestructura), cuándo se ejecuta (schedule), parámetros por defecto, y configuración de almacenamiento. Un Flow se puede ejecutar directamente como script Python, pero un Deployment permite orquestación programada, ejecución remota y gestión desde la UI de Prefect.

---

**2. ¿Qué es un Work Pool? ¿Para qué sirve?**

**Respuesta:** Un Work Pool es un grupo lógico de infraestructura donde pueden ejecutarse deployments. Define el tipo de infraestructura (Kubernetes, Docker, Cloud Run, Process, etc.) y sus configuraciones. Sirve para: (1) abstraer la infraestructura del código del flow, (2) permitir que múltiples deployments compartan la misma configuración de infraestructura, (3) escalar dinámicamente workers según la carga, y (4) separar ambientes (dev, staging, prod). Los Work Pools envían trabajo a Workers que escuchan activamente.

---

**3. ¿Qué es un Worker? ¿Cómo se relaciona con el Work Pool?**

**Respuesta:** Un Worker es un proceso que se ejecuta en la infraestructura objetivo, escucha por trabajo del Work Pool, y ejecuta los flow runs cuando son programados. La relación es: Work Pool (configuración) → Worker (ejecutor) → Flow Run (ejecución). Un Work Pool puede tener múltiples Workers ejecutándose simultáneamente para procesamiento paralelo. El Worker "poll" el Work Pool, obtiene flow runs pendientes, los ejecuta en su entorno local (container, proceso, etc.), y reporta resultados a Prefect.


### 5.2 Scheduling

Investiguen las opciones de scheduling:

**1. ¿Qué tipos de schedules soporta Prefect? Describan al menos 3:**

| Tipo de Schedule     | Descripción                                                              | Ejemplo                                                                 |
| -------------------- | ------------------------------------------------------------------------ | ----------------------------------------------------------------------- |
| **CronSchedule**     | Usa sintaxis cron estándar para definir intervalos de tiempo             | `cron="0 6 * * *"` ejecuta diariamente a las 6 AM                       |
| **IntervalSchedule** | Ejecuta cada X tiempo desde un punto de inicio                           | `interval=timedelta(hours=3)` ejecuta cada 3 horas                      |
| **RRuleSchedule**    | Usa la especificación iCalendar RFC para reglas complejas de recurrencia | `rrule="FREQ=WEEKLY;BYDAY=MO,WE,FR"` ejecuta lunes, miércoles y viernes |

---

**2. ¿Cómo expresarían "ejecutar todos los días a las 6 AM" en cron?**

**Respuesta:** `0 6 * * *` donde cada campo representa: minuto (0), hora (6), día del mes (_), mes (_), día de la semana (\*). El asterisco significa "cada" o "cualquier" valor para ese campo.

---

**3. ¿Qué es RRuleSchedule? ¿Cuándo lo usarían sobre cron?**

**Respuesta:** RRuleSchedule usa la especificación RFC 5545 (iCalendar) para reglas de recurrencia complejas. Es más expresivo que cron para casos como: "cada primer lunes del mes", "cada día laboral excepto festivos", "cada 2 semanas los martes y jueves". Se prefiere sobre cron cuando necesitas: (1) lógica de calendario compleja (días laborales, fines de semana), (2) exclusiones específicas de fechas, (3) intervalos que cron no puede expresar naturalmente, o (4) mejor legibilidad para reglas complejas.


### 5.3 Crear un Deployment (conceptual)

Basándose en la documentación, escriban el código para crear un deployment de su flow:


In [12]:
# TODO: Completar basándose en la documentación de Deployments
# https://docs.prefect.io/latest/concepts/deployments/

# NOTA: El código de deployment con .serve() no se puede ejecutar directamente en notebooks
# debido a que crea un event loop que entra en conflicto con el event loop de Jupyter.
# Este código está comentado como referencia de cómo se haría en un script .py

"""
from prefect import flow

# Opción 1: Usando serve() - más simple (solo funciona en scripts .py)
if __name__ == "__main__":
    etl_flow.serve(
        name="etl-pipeline-deployment",  # nombre del deployment
        cron="0 6 * * *",  # schedule en formato cron
        tags=["etl", "produccion"],  # tags para organización
    )

# Opción 2: Usando deploy() - más control
# Requiere configuración de work pool
# etl_flow.deploy(
#     name="etl-pipeline",
#     work_pool_name="default-pool",
#     cron="0 6 * * *",
# )
"""

# Para crear deployments desde un notebook, usa el CLI en terminal:
# prefect deploy --name etl-pipeline --cron "0 6 * * *" --tag etl --tag produccion

print("⚠️ Los deployments con .serve() deben ejecutarse desde un script .py, no desde notebooks")
print("💡 Para crear un deployment desde terminal, usa:")
print("   prefect deploy etl_flow.py:etl_flow --name etl-pipeline --cron '0 6 * * *'")

⚠️ Los deployments con .serve() deben ejecutarse desde un script .py, no desde notebooks
💡 Para crear un deployment desde terminal, usa:
   prefect deploy etl_flow.py:etl_flow --name etl-pipeline --cron '0 6 * * *'


---

## Parte 6 — Extensión DataOps (15 min)

Elijan UNA extensión e impleméntenla. Deben incluir comentarios explicando qué hace cada parte basándose en la documentación que investigaron.

### Opción A — Validación con logging estructurado


In [13]:
from prefect import get_run_logger

@task(retries=2, retry_delay_seconds=5)  # agregar retries
def validate_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Valida la calidad de los datos.
    Usa logging estructurado de Prefect.
    """
    logger = get_run_logger()
    errors = []
    
    # TODO: Implementar validaciones con logging apropiado
    # Usar logger.info(), logger.warning(), logger.error()
    
    logger.info("Iniciando validación de datos")
    
    if len(df) <= 0:
        logger.error("DataFrame vacío detectado")
        errors.append("DataFrame vacío")
    
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        logger.warning(f"Valores nulos encontrados: {null_counts.to_dict()}")
    
    if errors:
        raise ValueError(f"Validación fallida: {errors}")
    
    logger.info("✅ Validación exitosa")
    return df

### Opción B — Flow parametrizado con caching


In [14]:
from datetime import timedelta

@task(cache_expiration=timedelta(minutes=15))  # cachear por N minutos
def extract_data_param(n_rows: int = 100):
    """Extract con caching - investigar cuándo se invalida el cache."""
    np.random.seed(42)
    
    data = {
        'fecha': pd.date_range(start='2024-01-01', periods=n_rows, freq='D'),
        'producto': np.random.choice(['A', 'B', 'C', 'D'], n_rows),
        'cantidad': np.random.randint(1, 50, n_rows),
        'precio_unitario': np.random.uniform(10, 100, n_rows).round(2),
        'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], n_rows)
    }
    
    df = pd.DataFrame(data)
    print(f"📥 Extraídos {len(df)} registros con parámetros")
    return df

@flow
def etl_flow_parametrized(
    min_amount: float = 0.0,
    output_path: str = "output.csv",
    n_rows: int = 100
):
    """
    Flow parametrizado.
    Investigar: ¿cómo afectan los parámetros al caching?
    """
    df_raw = extract_data_param(n_rows=n_rows)
    df_clean = transform_data(df_raw)
    
    # Filtrar por monto mínimo si se especifica
    if min_amount > 0:
        df_clean = df_clean[df_clean['total'] >= min_amount]
        print(f"Filtrados {len(df_clean)} registros con monto >= {min_amount}")
    
    load_data(df_clean, output_path=output_path)
    return df_clean

### Opción C — Pipeline con concurrencia


In [15]:
from prefect.task_runners import ConcurrentTaskRunner

@flow(task_runner=ConcurrentTaskRunner())
def etl_flow_concurrent():
    """
    Flow con procesamiento paralelo por región.
    Investigar: ¿cuándo es útil vs. secuencial?
    """
    # Extraer y transformar datos
    df_raw = extract_data()
    df_clean = transform_data(df_raw)
    
    # Procesar cada región en paralelo usando .submit()
    regiones = ['Norte', 'Sur', 'Este', 'Oeste']
    futures = [process_region.submit(r, df_clean) for r in regiones]
    
    # Esperar y recolectar resultados
    results = [f.result() for f in futures]
    
    # Guardar resultados agregados
    load_data(df_clean, output_path="output_concurrent.csv")
    
    print(f"📊 Procesamiento concurrente completado: {results}")
    return results

---

## Parte 7 — Reflexión y Conexión con DataOps (5 min)

### 7.1 Conceptos de Prefect

Basándose en su investigación, expliquen:

**1. ¿Cómo ayuda Prefect a implementar el principio de "Observabilidad" de DataOps?**

**Respuesta:** Prefect implementa observabilidad a través de: (1) **Logging estructurado automático**: cada task y flow registra su ejecución, estado, y mensajes con timestamps; (2) **UI en tiempo real**: visualización de DAGs, estados, y dependencias sin configuración adicional; (3) **Tracking de estados granulares**: Pending, Running, Completed, Failed, Cached, etc., permitiendo entender exactamente qué pasó; (4) **Result persistence**: almacena outputs intermedios para inspección post-ejecución; (5) **Métricas y alertas**: eventos automáticos que pueden disparar notificaciones. Esto permite detectar fallos rápidamente, debugear con contexto completo, y entender el comportamiento del pipeline sin instrumentación manual.

---

**2. ¿Cómo ayuda el caching a la "Reproducibilidad"?**

**Respuesta:** El caching mejora la reproducibilidad al: (1) **Garantizar determinismo**: con la misma cache key (mismos inputs), siempre se obtiene el mismo resultado sin re-ejecutar; (2) **Reducir variabilidad externa**: si una API o fuente de datos cambia, el cache preserva el estado anterior para comparaciones; (3) **Facilitar experimentación**: permite iterar en pasos posteriores del pipeline sin re-ejecutar pasos costosos anteriores que no cambiaron; (4) **Versionado implícito**: al cambiar inputs, se genera nueva cache, creando un historial de resultados; (5) **Debugging consistente**: permite reproducir exactamente una ejecución problemática usando los mismos datos cacheados.

---

**3. ¿Cómo conectan los Deployments con "CI/CD para datos"?**

**Respuesta:** Los Deployments habilitan CI/CD para datos al: (1) **Separar código de ejecución**: el código del flow vive en un repo Git, el deployment define cómo/cuándo se ejecuta; (2) **Promover entre ambientes**: mismo código, diferentes deployments para dev/staging/prod; (3) **Versionado automático**: cada cambio en el flow crea una nueva versión del deployment, permitiendo rollbacks; (4) **Integración con CI**: se pueden crear/actualizar deployments desde pipelines de CI (GitHub Actions, GitLab CI) automáticamente al hacer merge; (5) **Testing automatizado**: deployments de prueba pueden ejecutarse en cada PR antes de llegar a producción; (6) **Infrastructure as Code**: la configuración del deployment es código, se versiona y se revisa como cualquier otro cambio.


### 7.2 Comparación con alternativas

Investiguen brevemente (pueden usar la web):

**1. ¿Qué diferencias hay entre Prefect y Apache Airflow? Mencionen al menos 2.**

- **Diferencia 1: Filosofía de desarrollo** - Prefect usa Python nativo sin DSLs especiales (workflows son funciones Python estándar), mientras Airflow requiere definir DAGs explícitamente usando su API específica con operadores. Prefect infiere dependencias del flujo de datos, Airflow requiere declaraciones explícitas de dependencias con `>>` o `set_upstream()`.

- **Diferencia 2: Ejecución dinámica** - Prefect permite crear tasks y branches dinámicamente en runtime basándose en datos (ej: número de tasks determinado por resultados previos), mientras Airflow requiere que el DAG esté completamente definido antes de la ejecución (DAG estático). Prefect maneja mejor workflows con lógica condicional compleja y control de flujo nativo de Python.

---

**2. ¿Qué es Dagster? ¿En qué se diferencia de Prefect?**

**Respuesta:** Dagster es otro orquestador moderno enfocado en "data assets" (recursos de datos como tablas, modelos ML) en lugar de tasks. Se diferencia de Prefect en: (1) **Paradigma**: Dagster se centra en el linaje de datos y gestión de assets con un grafo de dependencias de assets, mientras Prefect se enfoca en la ejecución de workflows de tasks; (2) **Testing**: Dagster tiene un framework de testing más robusto para validar transformaciones de datos antes de producción; (3) **Type system**: Dagster tiene un sistema de tipos más estricto para inputs/outputs de assets; (4) **Target audience**: Dagster apunta más a equipos de Analytics Engineering y dbt users, Prefect a ingenieros de datos y MLOps generales. Ambos son modernos, Pythonic, y superiores a Airflow en usabilidad.


---

## Entregable

### 1. Código Prefect

- Pipeline base funcionando (@flow + @task)
- Al menos UNA extensión implementada (A, B o C)
- Comentarios explicando las decisiones basadas en la documentación

### 2. Documento de investigación

Incluir en el notebook o en un `.md` separado:

- Respuestas a todas las preguntas de investigación (Partes 1, 4, 5)
- Citas/referencias a la documentación oficial
- Reflexiones (Parte 7)

### 3. Evidencia de ejecución

Screenshots o logs mostrando:

- Ejecución exitosa del flow
- Logs de Prefect con estados de las tasks
- (Opcional) UI de Prefect si la exploraron


---

## Rúbrica

| Criterio                      | Peso | Descripción                                                                                                |
| ----------------------------- | ---- | ---------------------------------------------------------------------------------------------------------- |
| **Investigación documentada** | 30%  | Respuestas completas basadas en la documentación oficial. Se nota que leyeron y entendieron los conceptos. |
| **Implementación técnica**    | 40%  | Pipeline funciona correctamente. Extensión implementada usa las funcionalidades investigadas.              |
| **Conexión DataOps/ML**       | 20%  | Reflexiones muestran comprensión de cómo Prefect habilita principios de DataOps.                           |
| **Calidad del código**        | 10%  | Código limpio, comentado, con explicaciones de las decisiones tomadas.                                     |
